In [4]:
alpha_name = "AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 10

In [5]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [6]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   2%|█▉                                                                                       | 126/5840 [00:00<00:07, 736.43it/s]

['AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_7', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_8', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_6', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_1', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_0', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_4', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_9', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_3', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_5', 'AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_2']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:01<00:00, 559.89it/s]



IC table for feature "AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_7" (mean std IR):
+-------------------+------------------+------------------+------------------+------------------+------------------+
|      period       |  IC_AlphaNet_1d  |  IC_AlphaNet_2d  |  IC_AlphaNet_3d  |  IC_AlphaNet_4d  |  IC_AlphaNet_5d  |
+===================+==================+==================+==================+==================+==================+
| 20190111-20190630 |  4.52 5.07 0.89  |  4.92 6.41 0.77  |  5.83 7.16 0.81  |  5.71 7.00 0.82  |  5.77 7.55 0.76  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20190701-20191231 |  3.46 5.87 0.59  |  1.81 4.55 0.40  |  2.64 4.00 0.66  |  3.90 5.50 0.71  |  3.59 4.39 0.82  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20200101-20200630 |  4.19 5.85 0.71  |  5.09 6.78 0.75  |  5.05 6.15 0.82  |  5.89

...


AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_5 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     212     |   4.19   |   6.29    |    8.75    |   1.43   |    58.97    | 3.49(20190111-20190131) |  1.11(20190131)   |    8.59    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     215     |   3.55   |   3.95    |    6.90    |   1.77   |    5

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 633.34it/s]



IC table for feature "AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_2" (mean std IR):
+-------------------+------------------+------------------+------------------+------------------+------------------+
|      period       |  IC_AlphaNet_1d  |  IC_AlphaNet_2d  |  IC_AlphaNet_3d  |  IC_AlphaNet_4d  |  IC_AlphaNet_5d  |
+===================+==================+==================+==================+==================+==================+
| 20190104-20190630 |  3.74 5.25 0.71  |  4.40 5.37 0.82  |  5.02 5.62 0.89  |  5.52 6.34 0.87  |  5.53 6.77 0.82  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20190701-20191231 |  3.44 5.23 0.66  |  3.42 4.66 0.73  |  3.35 4.31 0.78  |  3.81 5.13 0.74  |  4.17 4.24 0.98  |
+-------------------+------------------+------------------+------------------+------------------+------------------+
| 20200101-20200630 |  4.31 5.34 0.81  |  4.59 6.26 0.73  |  4.67 5.69 0.82  |  4.89

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:06<00:00, 87.24it/s]



AlphaNet_8Input_Shift_1_Sequence_20_10d_ret_standscalar_2 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     211     |   6.03   |   6.39    |   12.71    |   1.98   |    61.54    | 3.73(20190109-20190131) |  1.43(20190429)   |   14.41    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     214     |   1.49   |   4.19    |    2.87    |   0.71   |    5